# PyCity Schools Analysis
- 
  From the district data we can see that for the 15 schools in the district, more students tend to pass reading (85.81%) than math (74.98%), with only 65.17% passing both subjects. 

  Looking further at data for individual schools, we can see that the highest performing school based off of overall student passing rates are Cabrera High School, Thomas High School, Griffin High School, Wilson High School, and Pena High School in that respective order. The poorest performing schools are Rodriguez High School, Figueroa High School, Huang High School, Hernandez High School, Johnson High School. It should be noted though that the poorer performing schools often have 3-4 times less students than those of the highest performing schools.
  
  Focusing on overall passing rate based off school size, it is evident that while there is not a significant difference in average scores for individual subjects, there is a significant increase in the passing rate for that of smaller to medium schools compared to that of large ones. 

  We can also see from scores based off school spending that spending both too little and too much per student is in correlation with lower overall passing rates. Spending over $1.70m per student causes scores to decrease drastically. 

  Finally, looking at scores by school type, we can see that charter schools overall performed oustandingly better than district schools. The passing rate for charter schools is approximately 40% higher than that of its district counterpart. 
  
---

In [307]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

# Rename columns to be more clear
rename_columns = {"student_name" : "Student Name", 
                  "gender" : "Gender", 
                  "grade" : "Grade", 
                  "school_name" : "School Name", 
                  "reading_score" : "Reading Score (%)", 
                  "math_score" : "Math Score (%)", 
                  "type" : "Type", 
                  "size" : "Size", 
                  "budget" : "Budget ($)"}


school_data_complete.rename(columns = rename_columns, inplace = True)
school_data_complete.head()

,Student ID,Student Name,Gender,Grade,School Name,Reading Score (%),Math Score (%),School ID,Type,Size,Budget ($)
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [308]:
# Calculate the total number of unique schools
school_count = school_data_complete["School Name"].nunique()
print(f"Number of unique schools: {school_count}")

Number of unique schools: 15


In [309]:
# Calculate the total number of students
student_count = school_data_complete.shape[0]
print(f"Total students: {student_count}")

Total students: 39170


In [310]:
# Calculate the total budget
total_budget = school_data_complete["Budget ($)"].sum()

# Format long integer to make it more legible
total_budget = '{:,}'.format(total_budget)
print(f"Total budget: ${total_budget}")

Total budget: $82,932,329,558


In [311]:
# Calculate the average (mean) math score
average_math_score = round(school_data_complete["Math Score (%)"].mean(), 2)
print(f"Average math score: {average_math_score}%")

Average math score: 78.99%


In [312]:
# Calculate the average (mean) reading score
average_reading_score = round(school_data_complete["Reading Score (%)"].mean(), 2)
print(f"Average reading score: {average_reading_score}%")

Average reading score: 81.88%


In [313]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["Math Score (%)"] >= 70)].count()["Student Name"]
passing_math_percentage = round(((passing_math_count / float(student_count)) * 100), 2)
print(f"% of students that passed math: {passing_math_percentage}%")

% of students that passed math: 74.98%


In [314]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete["Reading Score (%)"] >= 70)].count()["Student Name"]
passing_reading_percentage = round(((passing_reading_count / float(student_count)) * 100), 2)
print(f"% of students that passed reading: {passing_reading_percentage}%")

% of students that passed reading: 85.81%


In [315]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["Math Score (%)"] >= 70) & (school_data_complete["Reading Score (%)"] >= 70)
].count()["Student Name"]
overall_passing_rate = round(((passing_math_reading_count /  float(student_count)) * 100), 2)
print(f"% of students that passed both math & reading: {overall_passing_rate}%")

% of students that passed both math & reading: 65.17%


In [316]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = { 
    "Total Schools": [school_count], 
    "Total Students": [student_count],
    "Total Budget": [total_budget],
    "Average Math Score": [average_math_score],
    "Average Reading Score": [average_reading_score],
    "% Passing Math": [passing_math_percentage],
    "% Passing Reading": [passing_reading_percentage],
    "% Overall Passing": [overall_passing_rate]
}

district_summary_df = pd.DataFrame(district_summary)

# Formatting
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)

# First remove dollar signs and commas from strings in "Budget", convert to float (from above formatting)
district_summary_df['Total Budget'] = district_summary_df['Total Budget'].str.replace(',', '').str.replace('$', '').astype(float)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$82,932,329,558.00",78.99,81.88,74.98,85.81,65.17


## School Summary

In [317]:
# Use the code provided to select all of the school types
school_types = school_data_complete["Type"].unique()

print(school_types)

# Select all schools
school_names = school_data_complete["School Name"].unique()
display(school_names)

['District' 'Charter']


array(['Huang High School', 'Figueroa High School', 'Shelton High School',
       'Hernandez High School', 'Griffin High School',
       'Wilson High School', 'Cabrera High School', 'Bailey High School',
       'Holden High School', 'Pena High School', 'Wright High School',
       'Rodriguez High School', 'Johnson High School', 'Ford High School',
       'Thomas High School'], dtype=object)

In [318]:
# Calculate the total student count per school
per_school_counts = school_data_complete.groupby("Type")["Student ID"].size()
display(per_school_counts)

per_school_name_counts = school_data_complete.groupby("School Name")["Student ID"].size()
display(per_school_name_counts)


Type
Charter     12194
District    26976
Name: Student ID, dtype: int64

School Name
Bailey High School       4976
Cabrera High School      1858
Figueroa High School     2949
Ford High School         2739
Griffin High School      1468
Hernandez High School    4635
Holden High School        427
Huang High School        2917
Johnson High School      4761
Pena High School          962
Rodriguez High School    3999
Shelton High School      1761
Thomas High School       1635
Wilson High School       2283
Wright High School       1800
Name: Student ID, dtype: int64

In [319]:
# Calculate the total school budget and per capita spending per school

#Total school budget
#print(f"Total school budget based on school type: \n")

#per_school_budgets = school_data_complete.groupby("Type")["Budget ($)"].sum()
#per_school_budgets_formatted = per_school_budgets.apply(lambda x: '${:,.0f}'.format(x))
#print(per_school_budgets_formatted)

print(f"\n\nTotal budget per school: \n")
per_school_name_budgets = school_data_complete.groupby("School Name")["Budget ($)"].sum()
per_school_name_budgets_formatted = per_school_name_budgets.apply(lambda x: '${:,.0f}'.format(x))
print(per_school_name_budgets_formatted)

# Per capita spending 
#per_school_capita = per_school_budgets / per_school_counts
per_school_name_capita = per_school_name_budgets / per_school_name_counts

#print(f"\n\nPer capita spending based on school type: \n")
#per_school_capita = per_school_capita.apply(lambda x: '${:,.0f}'.format(x))
#display(per_school_capita)

print(f"\n\nPer capita spending per school: \n")
per_school_name_capita_formatted = per_school_name_capita.apply(lambda x: '${:,.0f}'.format(x))
display(per_school_name_capita_formatted)




Total budget per school: 

School Name
Bailey High School       $15,549,641,728
Cabrera High School       $2,009,159,448
Figueroa High School      $5,557,128,039
Ford High School          $4,831,365,924
Griffin High School       $1,346,890,000
Hernandez High School    $14,007,062,700
Holden High School          $105,933,149
Huang High School         $5,573,322,295
Johnson High School      $14,733,628,650
Pena High School            $563,595,396
Rodriguez High School    $10,186,904,637
Shelton High School       $1,860,672,600
Thomas High School        $1,705,517,550
Wilson High School        $3,012,587,442
Wright High School        $1,888,920,000
Name: Budget ($), dtype: object


Per capita spending per school: 



School Name
Bailey High School       $3,124,928
Cabrera High School      $1,081,356
Figueroa High School     $1,884,411
Ford High School         $1,763,916
Griffin High School        $917,500
Hernandez High School    $3,022,020
Holden High School         $248,087
Huang High School        $1,910,635
Johnson High School      $3,094,650
Pena High School           $585,858
Rodriguez High School    $2,547,363
Shelton High School      $1,056,600
Thomas High School       $1,043,130
Wilson High School       $1,319,574
Wright High School       $1,049,400
dtype: object

In [320]:
# Calculate the average test scores per school

#per_school_math = round((school_data_complete.groupby("Type")["Math Score (%)"].mean()), 2)
per_school_name_math = round((school_data_complete.groupby("School Name")["Math Score (%)"].mean()), 2)

#per_school_reading = round((school_data_complete.groupby("Type")["Reading Score (%)"].mean()), 2)
per_school_name_reading = round((school_data_complete.groupby("School Name")["Reading Score (%)"].mean()), 2)

#print(f"Average test scores per school type:\n")
#display(per_school_math)
#display(per_school_reading)

print(f"\n\nAverage test scores per school:\n")
display(per_school_name_math)
display(per_school_name_reading)




Average test scores per school:



School Name
Bailey High School       77.05
Cabrera High School      83.06
Figueroa High School     76.71
Ford High School         77.10
Griffin High School      83.35
Hernandez High School    77.29
Holden High School       83.80
Huang High School        76.63
Johnson High School      77.07
Pena High School         83.84
Rodriguez High School    76.84
Shelton High School      83.36
Thomas High School       83.42
Wilson High School       83.27
Wright High School       83.68
Name: Math Score (%), dtype: float64

School Name
Bailey High School       81.03
Cabrera High School      83.98
Figueroa High School     81.16
Ford High School         80.75
Griffin High School      83.82
Hernandez High School    80.93
Holden High School       83.81
Huang High School        81.18
Johnson High School      80.97
Pena High School         84.04
Rodriguez High School    80.74
Shelton High School      83.73
Thomas High School       83.85
Wilson High School       83.99
Wright High School       83.96
Name: Reading Score (%), dtype: float64

In [321]:
# Calculate the number of students per school with math scores of 70 or higher

#students_passing_math = school_data_complete[school_data_complete["Math Score (%)"] >= 70].groupby("Type").size()
#print(f"Number of students per school type with math scores 70 or higher:\n")
#display(students_passing_math)

students_passing_math_per_school = school_data_complete[school_data_complete["Math Score (%)"] >= 70].groupby("School Name").size()
print(f"\n\nNumber of students per school with math scores 70 or higher:\n")
display(students_passing_math_per_school)




Number of students per school with math scores 70 or higher:



School Name
Bailey High School       3318
Cabrera High School      1749
Figueroa High School     1946
Ford High School         1871
Griffin High School      1371
Hernandez High School    3094
Holden High School        395
Huang High School        1916
Johnson High School      3145
Pena High School          910
Rodriguez High School    2654
Shelton High School      1653
Thomas High School       1525
Wilson High School       2143
Wright High School       1680
dtype: int64

In [322]:
# Calculate the number of students per school with reading scores of 70 or higher

#students_passing_reading = school_data_complete[school_data_complete["Reading Score (%)"] >= 70].groupby("Type").size()
#print(f"Number of students per school type with reading scores 70 or higher:\n")
#display(students_passing_reading)

students_passing_reading_per_school = school_data_complete[school_data_complete["Reading Score (%)"] >= 70].groupby("School Name").size()
print(f"\n\nNumber of students per school with reading scores 70 or higher:\n")
display(students_passing_reading_per_school)




Number of students per school with reading scores 70 or higher:



School Name
Bailey High School       4077
Cabrera High School      1803
Figueroa High School     2381
Ford High School         2172
Griffin High School      1426
Hernandez High School    3748
Holden High School        411
Huang High School        2372
Johnson High School      3867
Pena High School          923
Rodriguez High School    3208
Shelton High School      1688
Thomas High School       1591
Wilson High School       2204
Wright High School       1739
dtype: int64

In [323]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete[
    (school_data_complete["Reading Score (%)"] >= 70) & (school_data_complete["Math Score (%)"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["School Name"]).size()

print(f"\nNumber of students per school passing both math & reading:")
display(school_students_passing_math_and_reading)


Number of students per school passing both math & reading:


School Name
Bailey High School       2719
Cabrera High School      1697
Figueroa High School     1569
Ford High School         1487
Griffin High School      1330
Hernandez High School    2481
Holden High School        381
Huang High School        1561
Johnson High School      2549
Pena High School          871
Rodriguez High School    2119
Shelton High School      1583
Thomas High School       1487
Wilson High School       2068
Wright High School       1626
dtype: int64

In [324]:
# Use the provided code to calculate the passing rates
per_school_passing_math = round(((students_passing_math_per_school / per_school_name_counts) * 100), 2)
per_school_passing_reading = round(((students_passing_reading_per_school / per_school_name_counts) * 100), 2)
overall_passing_rate = round(((school_students_passing_math_and_reading / per_school_name_counts) * 100), 2)

print(f"\nPercent of students passed math:")
display(per_school_passing_math)

print(f"\n\nPercent of students passed reading:")
display(per_school_passing_reading)

print(f"\n\nPercent of students that passed both:")
display(overall_passing_rate)


Percent of students passed math:


School Name
Bailey High School       66.68
Cabrera High School      94.13
Figueroa High School     65.99
Ford High School         68.31
Griffin High School      93.39
Hernandez High School    66.75
Holden High School       92.51
Huang High School        65.68
Johnson High School      66.06
Pena High School         94.59
Rodriguez High School    66.37
Shelton High School      93.87
Thomas High School       93.27
Wilson High School       93.87
Wright High School       93.33
dtype: float64



Percent of students passed reading:


School Name
Bailey High School       81.93
Cabrera High School      97.04
Figueroa High School     80.74
Ford High School         79.30
Griffin High School      97.14
Hernandez High School    80.86
Holden High School       96.25
Huang High School        81.32
Johnson High School      81.22
Pena High School         95.95
Rodriguez High School    80.22
Shelton High School      95.85
Thomas High School       97.31
Wilson High School       96.54
Wright High School       96.61
dtype: float64



Percent of students that passed both:


School Name
Bailey High School       54.64
Cabrera High School      91.33
Figueroa High School     53.20
Ford High School         54.29
Griffin High School      90.60
Hernandez High School    53.53
Holden High School       89.23
Huang High School        53.51
Johnson High School      53.54
Pena High School         90.54
Rodriguez High School    52.99
Shelton High School      89.89
Thomas High School       90.95
Wilson High School       90.58
Wright High School       90.33
dtype: float64

In [325]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = { 
    "Total Students": per_school_name_counts,
    "Total School Budget": per_school_name_budgets,
    "Per Student Budget": per_school_name_capita,
    "# of Students Passing Math": students_passing_math_per_school,
    "# of Students Passing Reading": students_passing_reading_per_school,
    "# of Students Passing Both": school_students_passing_math_and_reading,
    "% Passing Math": per_school_passing_math,
    "% Passing Reading": per_school_passing_reading,
    "% Overall Passing": overall_passing_rate
}


per_school_summary = pd.DataFrame(per_school_summary).reset_index()

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
display(per_school_summary)

,School Name,Total Students,Total School Budget,Per Student Budget,# of Students Passing Math,# of Students Passing Reading,# of Students Passing Both,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,4976,"$15,549,641,728.00","$3,124,928.00",3318,4077,2719,66.68,81.93,54.64
1,Cabrera High School,1858,"$2,009,159,448.00","$1,081,356.00",1749,1803,1697,94.13,97.04,91.33
2,Figueroa High School,2949,"$5,557,128,039.00","$1,884,411.00",1946,2381,1569,65.99,80.74,53.20
3,Ford High School,2739,"$4,831,365,924.00","$1,763,916.00",1871,2172,1487,68.31,79.30,54.29
4,Griffin High School,1468,"$1,346,890,000.00","$917,500.00",1371,1426,1330,93.39,97.14,90.60
5,Hernandez High School,4635,"$14,007,062,700.00","$3,022,020.00",3094,3748,2481,66.75,80.86,53.53
6,Holden High School,427,"$105,933,149.00","$248,087.00",395,411,381,92.51,96.25,89.23
7,Huang High School,2917,"$5,573,322,295.00","$1,910,635.00",1916,2372,1561,65.68,81.32,53.51
8,Johnson High School,4761,"$14,733,628,650.00","$3,094,650.00",3145,3867,2549,66.06,81.22,53.54
9,Pena High School,962,"$563,595,396.00","$585,858.00",910,923,871,94.59,95.95,90.54


## Highest-Performing Schools (by % Overall Passing)

In [326]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values(by="% Overall Passing", ascending=False)
top_schools.head(5)

,School Name,Total Students,Total School Budget,Per Student Budget,# of Students Passing Math,# of Students Passing Reading,# of Students Passing Both,% Passing Math,% Passing Reading,% Overall Passing
1,Cabrera High School,1858,"$2,009,159,448.00","$1,081,356.00",1749,1803,1697,94.13,97.04,91.33
12,Thomas High School,1635,"$1,705,517,550.00","$1,043,130.00",1525,1591,1487,93.27,97.31,90.95
4,Griffin High School,1468,"$1,346,890,000.00","$917,500.00",1371,1426,1330,93.39,97.14,90.60
13,Wilson High School,2283,"$3,012,587,442.00","$1,319,574.00",2143,2204,2068,93.87,96.54,90.58
9,Pena High School,962,"$563,595,396.00","$585,858.00",910,923,871,94.59,95.95,90.54


## Bottom Performing Schools (By % Overall Passing)

In [327]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values(by="% Overall Passing", ascending=True)
bottom_schools.head(5)

,School Name,Total Students,Total School Budget,Per Student Budget,# of Students Passing Math,# of Students Passing Reading,# of Students Passing Both,% Passing Math,% Passing Reading,% Overall Passing
10,Rodriguez High School,3999,"$10,186,904,637.00","$2,547,363.00",2654,3208,2119,66.37,80.22,52.99
2,Figueroa High School,2949,"$5,557,128,039.00","$1,884,411.00",1946,2381,1569,65.99,80.74,53.20
7,Huang High School,2917,"$5,573,322,295.00","$1,910,635.00",1916,2372,1561,65.68,81.32,53.51
5,Hernandez High School,4635,"$14,007,062,700.00","$3,022,020.00",3094,3748,2481,66.75,80.86,53.53
8,Johnson High School,4761,"$14,733,628,650.00","$3,094,650.00",3145,3867,2549,66.06,81.22,53.54


## Math Scores by Grade

In [328]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["Grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["Grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["Grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["Grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_math_scores = round((ninth_graders.groupby("School Name")["Math Score (%)"].mean()), 2)
tenth_grader_math_scores = round((tenth_graders.groupby("School Name")["Math Score (%)"].mean()), 2)
eleventh_grader_math_scores = round((eleventh_graders.groupby("School Name")["Math Score (%)"].mean()), 2)
twelfth_grader_math_scores = round((twelfth_graders.groupby("School Name")["Math Score (%)"].mean()), 2)

#display(ninth_grade_math_scores)
# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.concat([ninth_grade_math_scores, tenth_grader_math_scores, eleventh_grader_math_scores, twelfth_grader_math_scores], axis = 1)
math_scores_by_grade = math_scores_by_grade.set_axis([ "% Average Math Score (Grade 9)","% Average Math Score (Grade 10)","% Average Math Score (Grade 11)","% Average Math Score (Grade 12)"], axis='columns')

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

,% Average Math Score (Grade 9),% Average Math Score (Grade 10),% Average Math Score (Grade 11),% Average Math Score (Grade 12)
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


## Reading Score by Grade 

In [329]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["Grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["Grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["Grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["Grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grade_reading_scores = round((ninth_graders.groupby("School Name")["Reading Score (%)"].mean()), 2)
tenth_grader_reading_scores = round((tenth_graders.groupby("School Name")["Reading Score (%)"].mean()), 2)
eleventh_grader_reading_scores = round((eleventh_graders.groupby("School Name")["Reading Score (%)"].mean()), 2)
twelfth_grader_reading_scores = round((twelfth_graders.groupby("School Name")["Reading Score (%)"].mean()), 2)

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.concat([ninth_grade_reading_scores, tenth_grader_reading_scores, eleventh_grader_reading_scores, twelfth_grader_reading_scores], axis = 1)
reading_scores_by_grade = math_scores_by_grade.set_axis([ "% Average Reading Score (Grade 9)","% Average Reading Score (Grade 10)","% Average Reading Score (Grade 11)","% Average Reading Score (Grade 12)"], axis='columns')

# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["% Average Reading Score (Grade 9)","% Average Reading Score (Grade 10)","% Average Reading Score (Grade 11)","% Average Reading Score (Grade 12)"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

,% Average Reading Score (Grade 9),% Average Reading Score (Grade 10),% Average Reading Score (Grade 11),% Average Reading Score (Grade 12)
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


## Scores by School Spending

In [330]:
# Establish the bins 
#spending_bins = [0, 585, 630, 645, 680]
#labels = ["<$585", "$585-630", "$630-645", "$645-680"]
spending_bins = [0, 250000, 975000, 1700000, 3150000]
labels = ["<$250k", "$250k-975k", "$975k-1.70m", "$1.70m-3.15m"]

In [331]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()
display(school_spending_df.head())
display(school_spending_df.dtypes)

,School Name,Total Students,Total School Budget,Per Student Budget,# of Students Passing Math,# of Students Passing Reading,# of Students Passing Both,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,4976,"$15,549,641,728.00","$3,124,928.00",3318,4077,2719,66.68,81.93,54.64
1,Cabrera High School,1858,"$2,009,159,448.00","$1,081,356.00",1749,1803,1697,94.13,97.04,91.33
2,Figueroa High School,2949,"$5,557,128,039.00","$1,884,411.00",1946,2381,1569,65.99,80.74,53.20
3,Ford High School,2739,"$4,831,365,924.00","$1,763,916.00",1871,2172,1487,68.31,79.30,54.29
4,Griffin High School,1468,"$1,346,890,000.00","$917,500.00",1371,1426,1330,93.39,97.14,90.60


School Name                       object
Total Students                     int64
Total School Budget               object
Per Student Budget                object
# of Students Passing Math         int64
# of Students Passing Reading      int64
# of Students Passing Both         int64
% Passing Math                   float64
% Passing Reading                float64
% Overall Passing                float64
dtype: object

In [332]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Per Student Budget"] = school_spending_df["Per Student Budget"].replace('[\$,]', '', regex=True).astype(float)
school_spending_df["Per Student Budget"] = school_spending_df["Per Student Budget"].astype(np.int64)
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget"], bins = spending_bins, labels = labels)
school_spending_df = school_spending_df.sort_values(by="Per Student Budget", ascending=True)
school_spending_df["Per Student Budget"] = school_spending_df["Per Student Budget"].map('${:,.2f}'.format)
display(school_spending_df)


,School Name,Total Students,Total School Budget,Per Student Budget,# of Students Passing Math,# of Students Passing Reading,# of Students Passing Both,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
6,Holden High School,427,"$105,933,149.00","$248,087.00",395,411,381,92.51,96.25,89.23,<$250k
9,Pena High School,962,"$563,595,396.00","$585,858.00",910,923,871,94.59,95.95,90.54,$250k-975k
4,Griffin High School,1468,"$1,346,890,000.00","$917,500.00",1371,1426,1330,93.39,97.14,90.60,$250k-975k
12,Thomas High School,1635,"$1,705,517,550.00","$1,043,130.00",1525,1591,1487,93.27,97.31,90.95,$975k-1.70m
14,Wright High School,1800,"$1,888,920,000.00","$1,049,400.00",1680,1739,1626,93.33,96.61,90.33,$975k-1.70m
11,Shelton High School,1761,"$1,860,672,600.00","$1,056,600.00",1653,1688,1583,93.87,95.85,89.89,$975k-1.70m
1,Cabrera High School,1858,"$2,009,159,448.00","$1,081,356.00",1749,1803,1697,94.13,97.04,91.33,$975k-1.70m
13,Wilson High School,2283,"$3,012,587,442.00","$1,319,574.00",2143,2204,2068,93.87,96.54,90.58,$975k-1.70m
3,Ford High School,2739,"$4,831,365,924.00","$1,763,916.00",1871,2172,1487,68.31,79.30,54.29,$1.70m-3.15m
2,Figueroa High School,2949,"$5,557,128,039.00","$1,884,411.00",1946,2381,1569,65.99,80.74,53.20,$1.70m-3.15m


In [333]:
# Create a Dataframe for average math scores by school
school_data_complete_1 = round(pd.DataFrame(school_data_complete.groupby("School Name")["Math Score (%)"].mean()), 2)

# Merge with our school_spending_df Dataframe
school_spending_df_1 = pd.merge(school_spending_df, school_data_complete_1, on= "School Name")

# Create a Dataframe for average reading scores by school
school_data_complete_2 = round(pd.DataFrame(school_data_complete.groupby("School Name")["Reading Score (%)"].mean()), 2)

# Merge with our school_spending_df_1 Dataframe
school_spending_df_2 = pd.merge(school_spending_df_1, school_data_complete_2, on= "School Name")
display(school_spending_df_2)


,School Name,Total Students,Total School Budget,Per Student Budget,# of Students Passing Math,# of Students Passing Reading,# of Students Passing Both,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student),Math Score (%),Reading Score (%)
0,Holden High School,427,"$105,933,149.00","$248,087.00",395,411,381,92.51,96.25,89.23,<$250k,83.80,83.81
1,Pena High School,962,"$563,595,396.00","$585,858.00",910,923,871,94.59,95.95,90.54,$250k-975k,83.84,84.04
2,Griffin High School,1468,"$1,346,890,000.00","$917,500.00",1371,1426,1330,93.39,97.14,90.60,$250k-975k,83.35,83.82
3,Thomas High School,1635,"$1,705,517,550.00","$1,043,130.00",1525,1591,1487,93.27,97.31,90.95,$975k-1.70m,83.42,83.85
4,Wright High School,1800,"$1,888,920,000.00","$1,049,400.00",1680,1739,1626,93.33,96.61,90.33,$975k-1.70m,83.68,83.96
5,Shelton High School,1761,"$1,860,672,600.00","$1,056,600.00",1653,1688,1583,93.87,95.85,89.89,$975k-1.70m,83.36,83.73
6,Cabrera High School,1858,"$2,009,159,448.00","$1,081,356.00",1749,1803,1697,94.13,97.04,91.33,$975k-1.70m,83.06,83.98
7,Wilson High School,2283,"$3,012,587,442.00","$1,319,574.00",2143,2204,2068,93.87,96.54,90.58,$975k-1.70m,83.27,83.99
8,Ford High School,2739,"$4,831,365,924.00","$1,763,916.00",1871,2172,1487,68.31,79.30,54.29,$1.70m-3.15m,77.10,80.75
9,Figueroa High School,2949,"$5,557,128,039.00","$1,884,411.00",1946,2381,1569,65.99,80.74,53.20,$1.70m-3.15m,76.71,81.16


In [334]:
#  Calculate averages for the desired columns. 
spending_math_scores = round(school_spending_df_2.groupby(["Spending Ranges (Per Student)"])["Math Score (%)"].mean(), 2)

spending_reading_scores = round(school_spending_df_2.groupby(["Spending Ranges (Per Student)"])["Reading Score (%)"].mean(), 2)

spending_passing_math = round(school_spending_df_2.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean(), 2)

spending_passing_reading = round(school_spending_df_2.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean(), 2)

overall_passing_spending = round(school_spending_df_2.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean(), 2)


/var/folders/qq/8hq4nv8x5rz6hyggl1ylc_kh0000gn/T/ipykernel_69955/2854135235.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_math_scores = round(school_spending_df_2.groupby(["Spending Ranges (Per Student)"])["Math Score (%)"].mean(), 2)
/var/folders/qq/8hq4nv8x5rz6hyggl1ylc_kh0000gn/T/ipykernel_69955/2854135235.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_reading_scores = round(school_spending_df_2.groupby(["Spending Ranges (Per Student)"])["Reading Score (%)"].mean(), 2)
/var/folders/qq/8hq4nv8x5rz6hyggl1ylc_kh0000gn/T/ipykernel_69955/2854135235.py:6: FutureWarning: The default o

In [335]:
# Assemble into DataFrame
spending_summary_df = pd.DataFrame({
    "Average Math Score (%)": spending_math_scores,
    "Average Reading Score (%)": spending_reading_scores,
    "% Passing Math": spending_passing_math,
    "% Passing Reading": spending_passing_reading,
    "% Overall Passing": overall_passing_spending
    }
)

# Display results
spending_summary_df

,Average Math Score (%),Average Reading Score (%),% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$250k,83.80,83.81,92.51,96.25,89.23
$250k-975k,83.60,83.93,93.99,96.54,90.57
$975k-1.70m,83.36,83.90,93.69,96.67,90.62
$1.70m-3.15m,76.96,80.97,66.55,80.80,53.67


## Scores by School Size

In [336]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [337]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"], bins = size_bins, labels = labels)
per_school_summary

,School Name,Total Students,Total School Budget,Per Student Budget,# of Students Passing Math,# of Students Passing Reading,# of Students Passing Both,% Passing Math,% Passing Reading,% Overall Passing,School Size
0,Bailey High School,4976,"$15,549,641,728.00","$3,124,928.00",3318,4077,2719,66.68,81.93,54.64,Large (2000-5000)
1,Cabrera High School,1858,"$2,009,159,448.00","$1,081,356.00",1749,1803,1697,94.13,97.04,91.33,Medium (1000-2000)
2,Figueroa High School,2949,"$5,557,128,039.00","$1,884,411.00",1946,2381,1569,65.99,80.74,53.20,Large (2000-5000)
3,Ford High School,2739,"$4,831,365,924.00","$1,763,916.00",1871,2172,1487,68.31,79.30,54.29,Large (2000-5000)
4,Griffin High School,1468,"$1,346,890,000.00","$917,500.00",1371,1426,1330,93.39,97.14,90.60,Medium (1000-2000)
5,Hernandez High School,4635,"$14,007,062,700.00","$3,022,020.00",3094,3748,2481,66.75,80.86,53.53,Large (2000-5000)
6,Holden High School,427,"$105,933,149.00","$248,087.00",395,411,381,92.51,96.25,89.23,Small (<1000)
7,Huang High School,2917,"$5,573,322,295.00","$1,910,635.00",1916,2372,1561,65.68,81.32,53.51,Large (2000-5000)
8,Johnson High School,4761,"$14,733,628,650.00","$3,094,650.00",3145,3867,2549,66.06,81.22,53.54,Large (2000-5000)
9,Pena High School,962,"$563,595,396.00","$585,858.00",910,923,871,94.59,95.95,90.54,Small (<1000)


In [338]:
# Merge per_school_summary & school_spending_df_2
per_school_summary_1 = pd.merge(per_school_summary, school_spending_df_2, on = "School Name")
display(per_school_summary_1)

,School Name,Total Students_x,Total School Budget_x,Per Student Budget_x,# of Students Passing Math_x,# of Students Passing Reading_x,# of Students Passing Both_x,% Passing Math_x,% Passing Reading_x,% Overall Passing_x,...,Per Student Budget_y,# of Students Passing Math_y,# of Students Passing Reading_y,# of Students Passing Both_y,% Passing Math_y,% Passing Reading_y,% Overall Passing_y,Spending Ranges (Per Student),Math Score (%),Reading Score (%)
0,Bailey High School,4976,"$15,549,641,728.00","$3,124,928.00",3318,4077,2719,66.68,81.93,54.64,...,"$3,124,928.00",3318,4077,2719,66.68,81.93,54.64,$1.70m-3.15m,77.05,81.03
1,Cabrera High School,1858,"$2,009,159,448.00","$1,081,356.00",1749,1803,1697,94.13,97.04,91.33,...,"$1,081,356.00",1749,1803,1697,94.13,97.04,91.33,$975k-1.70m,83.06,83.98
2,Figueroa High School,2949,"$5,557,128,039.00","$1,884,411.00",1946,2381,1569,65.99,80.74,53.20,...,"$1,884,411.00",1946,2381,1569,65.99,80.74,53.20,$1.70m-3.15m,76.71,81.16
3,Ford High School,2739,"$4,831,365,924.00","$1,763,916.00",1871,2172,1487,68.31,79.30,54.29,...,"$1,763,916.00",1871,2172,1487,68.31,79.30,54.29,$1.70m-3.15m,77.10,80.75
4,Griffin High School,1468,"$1,346,890,000.00","$917,500.00",1371,1426,1330,93.39,97.14,90.60,...,"$917,500.00",1371,1426,1330,93.39,97.14,90.60,$250k-975k,83.35,83.82
5,Hernandez High School,4635,"$14,007,062,700.00","$3,022,020.00",3094,3748,2481,66.75,80.86,53.53,...,"$3,022,020.00",3094,3748,2481,66.75,80.86,53.53,$1.70m-3.15m,77.29,80.93
6,Holden High School,427,"$105,933,149.00","$248,087.00",395,411,381,92.51,96.25,89.23,...,"$248,087.00",395,411,381,92.51,96.25,89.23,<$250k,83.80,83.81
7,Huang High School,2917,"$5,573,322,295.00","$1,910,635.00",1916,2372,1561,65.68,81.32,53.51,...,"$1,910,635.00",1916,2372,1561,65.68,81.32,53.51,$1.70m-3.15m,76.63,81.18
8,Johnson High School,4761,"$14,733,628,650.00","$3,094,650.00",3145,3867,2549,66.06,81.22,53.54,...,"$3,094,650.00",3145,3867,2549,66.06,81.22,53.54,$1.70m-3.15m,77.07,80.97
9,Pena High School,962,"$563,595,396.00","$585,858.00",910,923,871,94.59,95.95,90.54,...,"$585,858.00",910,923,871,94.59,95.95,90.54,$250k-975k,83.84,84.04


In [339]:
# Calculate averages for the desired columns. 
size_math_scores = round(per_school_summary_1.groupby(["School Size"])["Math Score (%)"].mean(), 2)
size_reading_scores = round(per_school_summary_1.groupby(["School Size"])["Reading Score (%)"].mean(), 2)
size_passing_math = round(per_school_summary_1.groupby(["School Size"])["% Passing Math_x"].mean(), 2)
size_passing_reading = round(per_school_summary_1.groupby(["School Size"])["% Passing Reading_x"].mean(), 2)
size_overall_passing = round(per_school_summary_1.groupby(["School Size"])["% Overall Passing_x"].mean(), 2)

display(size_math_scores, size_reading_scores, size_passing_math, size_passing_reading, size_overall_passing)

/var/folders/qq/8hq4nv8x5rz6hyggl1ylc_kh0000gn/T/ipykernel_69955/3088402709.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_math_scores = round(per_school_summary_1.groupby(["School Size"])["Math Score (%)"].mean(), 2)
/var/folders/qq/8hq4nv8x5rz6hyggl1ylc_kh0000gn/T/ipykernel_69955/3088402709.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_reading_scores = round(per_school_summary_1.groupby(["School Size"])["Reading Score (%)"].mean(), 2)
/var/folders/qq/8hq4nv8x5rz6hyggl1ylc_kh0000gn/T/ipykernel_69955/3088402709.py:4: FutureWarning: The default of observed=False is deprecated and will be c

School Size
Small (<1000)         83.82
Medium (1000-2000)    83.37
Large (2000-5000)     77.74
Name: Math Score (%), dtype: float64

School Size
Small (<1000)         83.93
Medium (1000-2000)    83.87
Large (2000-5000)     81.34
Name: Reading Score (%), dtype: float64

School Size
Small (<1000)         93.55
Medium (1000-2000)    93.60
Large (2000-5000)     69.96
Name: % Passing Math_x, dtype: float64

School Size
Small (<1000)         96.10
Medium (1000-2000)    96.79
Large (2000-5000)     82.77
Name: % Passing Reading_x, dtype: float64

School Size
Small (<1000)         89.88
Medium (1000-2000)    90.62
Large (2000-5000)     58.28
Name: % Overall Passing_x, dtype: float64

In [340]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame({
    "Average Math Score (%)": size_math_scores,
    "Average Reading Score (%)": size_reading_scores,
    "% Passing Math": size_passing_math,
    "% Passing Reading": size_passing_reading,
    "% Overall Passing": size_overall_passing
})

# Display results
size_summary

,Average Math Score (%),Average Reading Score (%),% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,89.88
Medium (1000-2000),83.37,83.87,93.60,96.79,90.62
Large (2000-5000),77.74,81.34,69.96,82.77,58.28


## Scores by School Type

In [341]:
# Let's locate the Type column from our complete school data dataframe
type_column = school_data_complete.loc[:, ["School Name", "Type"]]
display(type_column)

# Merge with our per_school_summary
per_school_summary_type = pd.merge(per_school_summary_1, type_column, on = "School Name")
display(per_school_summary_type)

,School Name,Type
0,Huang High School,District
1,Huang High School,District
2,Huang High School,District
3,Huang High School,District
4,Huang High School,District
...,...,...
39165,Thomas High School,Charter
39166,Thomas High School,Charter
39167,Thomas High School,Charter
39168,Thomas High School,Charter


,School Name,Total Students_x,Total School Budget_x,Per Student Budget_x,# of Students Passing Math_x,# of Students Passing Reading_x,# of Students Passing Both_x,% Passing Math_x,% Passing Reading_x,% Overall Passing_x,...,# of Students Passing Math_y,# of Students Passing Reading_y,# of Students Passing Both_y,% Passing Math_y,% Passing Reading_y,% Overall Passing_y,Spending Ranges (Per Student),Math Score (%),Reading Score (%),Type
0,Bailey High School,4976,"$15,549,641,728.00","$3,124,928.00",3318,4077,2719,66.68,81.93,54.64,...,3318,4077,2719,66.68,81.93,54.64,$1.70m-3.15m,77.05,81.03,District
1,Bailey High School,4976,"$15,549,641,728.00","$3,124,928.00",3318,4077,2719,66.68,81.93,54.64,...,3318,4077,2719,66.68,81.93,54.64,$1.70m-3.15m,77.05,81.03,District
2,Bailey High School,4976,"$15,549,641,728.00","$3,124,928.00",3318,4077,2719,66.68,81.93,54.64,...,3318,4077,2719,66.68,81.93,54.64,$1.70m-3.15m,77.05,81.03,District
3,Bailey High School,4976,"$15,549,641,728.00","$3,124,928.00",3318,4077,2719,66.68,81.93,54.64,...,3318,4077,2719,66.68,81.93,54.64,$1.70m-3.15m,77.05,81.03,District
4,Bailey High School,4976,"$15,549,641,728.00","$3,124,928.00",3318,4077,2719,66.68,81.93,54.64,...,3318,4077,2719,66.68,81.93,54.64,$1.70m-3.15m,77.05,81.03,District
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39165,Wright High School,1800,"$1,888,920,000.00","$1,049,400.00",1680,1739,1626,93.33,96.61,90.33,...,1680,1739,1626,93.33,96.61,90.33,$975k-1.70m,83.68,83.96,Charter
39166,Wright High School,1800,"$1,888,920,000.00","$1,049,400.00",1680,1739,1626,93.33,96.61,90.33,...,1680,1739,1626,93.33,96.61,90.33,$975k-1.70m,83.68,83.96,Charter
39167,Wright High School,1800,"$1,888,920,000.00","$1,049,400.00",1680,1739,1626,93.33,96.61,90.33,...,1680,1739,1626,93.33,96.61,90.33,$975k-1.70m,83.68,83.96,Charter
39168,Wright High School,1800,"$1,888,920,000.00","$1,049,400.00",1680,1739,1626,93.33,96.61,90.33,...,1680,1739,1626,93.33,96.61,90.33,$975k-1.70m,83.68,83.96,Charter


In [342]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = round(per_school_summary_type.groupby(["Type"])["Math Score (%)"].mean(), 2)
average_reading_score_by_type = round(per_school_summary_type.groupby(["Type"])["Reading Score (%)"].mean(), 2)
average_percent_passing_math_by_type = round(per_school_summary_type.groupby(["Type"])["% Passing Math_x"].mean(), 2)
average_percent_passing_reading_by_type = round(per_school_summary_type.groupby(["Type"])["% Passing Reading_x"].mean(), 2)
average_percent_overall_passing_by_type = round(per_school_summary_type.groupby(["Type"])["% Overall Passing_x"].mean(), 2)

In [343]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame({
    "Average Math Score (%)": average_math_score_by_type,
    "Average Reading Score (%)": average_reading_score_by_type,
    "% Passing Math": average_percent_passing_math_by_type,
    "% Passing Reading": average_percent_passing_reading_by_type,
    "% Overall Passing": average_percent_overall_passing_by_type
})

# Display results
type_summary

,Average Math Score (%),Average Reading Score (%),% Passing Math,% Passing Reading,% Overall Passing
Type,,,,,
Charter,83.40,83.90,93.70,96.65,90.56
District,76.99,80.96,66.52,80.90,53.70
